## Blending Degrader demo

author: Shuang Liang

This notebook demonstrate the use of `rail.creation.degradation.unrec_bl_model`, which uses Friends of Friends to finds sources close to each other and merge them into unrecognized blends

In [ ]:
from rail.creation.degraders.unrec_bl_model import UnrecBlModel

from rail.core.data import PqHandle
from rail.core.stage import RailStage

import matplotlib.pyplot as plt
import pandas as pd, numpy as np

In [ ]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True


### Create a random catalog with ugrizy+YJHF bands as the the true input

In [ ]:
data = np.random.normal(23, 3, size = (1000,12))
data[:, 0] = np.random.uniform(low=0, high=0.03, size=1000)
data[:, 1] = np.random.uniform(low=0, high=0.03, size=1000)

data_df = pd.DataFrame(data=data,    # values
            columns=['ra', 'dec', 'u', 'g', 'r', 'i', 'z', 'y', 'Y', 'J', 'H', 'F'])

data_truth = DS.add_data('input', data_df, PqHandle)

In [ ]:
#data_df.to_parquet('bl_test.pq')

In [ ]:
plt.scatter(data_truth.data['ra'], data_truth.data['dec'], s=5)
plt.xlabel("Ra [Deg]", fontsize=14)
plt.ylabel("Dec [Deg]", fontsize=14)
plt.show()


### The blending model

In [ ]:
## radius cut is in arcsecs
blModel = UnrecBlModel.make_stage(name='unrec_bl_model', ra_label='ra', dec_label='dec', linking_lengths=1.0, \
                                  bands='ugrizyYJHF')
blModel.get_config_dict()


In [ ]:
samples_w_bl = blModel(data_truth)
samples_w_bl()


In [ ]:
fig, ax = plt.subplots(figsize=(6, 5), dpi=100)

ax.scatter(data_truth.data['ra'],   data_truth.data['dec'],   s=10, facecolors='none', edgecolors='b', label='Original')
ax.scatter(samples_w_bl.data['ra'], samples_w_bl.data['dec'], s=5, c='r', label='w. Unrec-BL')

ax.legend(loc=2, fontsize=12)
ax.set_xlabel("Ra [Deg]", fontsize=14)
ax.set_ylabel("Dec [Deg]", fontsize=14)

plt.show()


In [ ]:
b = 'r'
plt.hist(data_truth.data[b], bins=np.linspace(10, 30, 20), label='Original')
plt.hist(samples_w_bl.data[b], bins=np.linspace(10, 30, 20),  fill=False, label='w. Unrec-BL')

plt.xlabel(fr'Magnitude ${b}$', fontsize=14)
plt.legend(fontsize=12)
plt.show()


In [ ]:
flux = 10**(-(data_truth.data[b]-28.10)/2.5)       # r band zp for lsst is 28.10
flux_bl = 10**(-(samples_w_bl.data[b]-28.10)/2.5)

plt.hist(flux, bins=np.linspace(0, 10000, 40), label='Original')
plt.hist(flux_bl, bins=np.linspace(0, 10000, 40), fill=False, label='w. Unrec-BL')

plt.xlabel(fr'Flux ${b}$', fontsize=14)
plt.yscale('log')
plt.legend(fontsize=12)
plt.show()
